### Mode Choice Model (HBW) - MNL

In [236]:
import pandas as pd
import numpy as np
import os
import larch
import caliperpy
from larch.roles import P,X
import subprocess
pd.set_option('display.max_columns', None)

In [237]:
# Open all files from appropriate folders
tazFile = "C:\\projects\\Oahu\\repo_new_model\\scenarios\\base_2022\\Output\\sedata\\scenario_se.bin"
survey_path = "C:\\projects\\Oahu\\repo_new_model\\docs\\data\\_private\\vis_survey\\output\\"
tripFile = survey_path + "vis_trips.csv"

skim_path = "C:\\projects\\Oahu\\repo_new_model\\docs\\data\\input\\skims\\"
autoSkim = skim_path + "Highway_AM.omx"
ptwalkSkim = skim_path + 'Walk_to_Bus.omx'
ptdriveSkim = skim_path + 'Drive_to_Bus.omx'
walkSkim = skim_path + 'Walk.omx'
bikeSkim = skim_path + 'Bike.omx'

In [238]:
dk = caliperpy.TransCAD.connect()
tazDF = dk.GetDataFrameFromBin(tazFile)
caliperpy.TransCAD.disconnect()

tripDF = pd.read_csv(tripFile)

Connecting to TransCAD...
Disconnected from TransCAD!


In [239]:
# Open omx file(s)
autoSkimFile = larch.OMX(autoSkim)
ptwalkSkimFile = larch.OMX(ptwalkSkim)
# ptdriveSkimFile = larch.OMX(ptdriveSkim)
walkSkimFile = larch.OMX(walkSkim)
bikeSkimFile = larch.OMX(bikeSkim)

zoneIDs = autoSkimFile.TAZ[:]

taz_alt_map = {zoneIDs[i]: i for i in range(len(zoneIDs))}

In [240]:
# Select only the relevant recorsds
estDF = tripDF[(tripDF["mode"] != "other") & (tripDF["mode"].notnull())]
# print(pd.unique(estDF["mode"]))

In [241]:
# Create the intrazonal and business indicators
estDF['IZ'] = estDF['iz_flag']
estDF['business_flag'] = np.select([estDF['visitor_type'] == "business"], [1], 0)

C:\Users\Kyle\AppData\Local\Temp\ipykernel_48376\4080613111.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  estDF['IZ'] = estDF['iz_flag']
C:\Users\Kyle\AppData\Local\Temp\ipykernel_48376\4080613111.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  estDF['business_flag'] = np.select([estDF['visitor_type'] == "business"], [1], 0)


In [242]:
# Attach the (free-flow) skims based on Orig_TAZ and Dest_TAZ
estDF['OTAZIndex'] = estDF['o_taz'].map(taz_alt_map)
origIdxs = estDF['OTAZIndex'].to_numpy()

estDF['DTAZIndex'] = estDF['d_taz'].map(taz_alt_map)
destIdxs = estDF['DTAZIndex'].to_numpy()

C:\Users\Kyle\AppData\Local\Temp\ipykernel_48376\3995184942.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  estDF['OTAZIndex'] = estDF['o_taz'].map(taz_alt_map)
C:\Users\Kyle\AppData\Local\Temp\ipykernel_48376\3995184942.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  estDF['DTAZIndex'] = estDF['d_taz'].map(taz_alt_map)


In [243]:
# Highway skims
estDF['AutoTime'] = pd.DataFrame(data = autoSkimFile['Time'][:][origIdxs, destIdxs], index = estDF.index)
estDF['DriveDist'] = pd.DataFrame(data = autoSkimFile['Distance'][:][origIdxs, destIdxs], index = estDF.index)

# Walk skims
estDF['WalkTime'] = pd.DataFrame(data = walkSkimFile['Time'][:][origIdxs, destIdxs], index = estDF.index)
estDF['WalkDist'] = pd.DataFrame(data = walkSkimFile['Distance'][:][origIdxs, destIdxs], index = estDF.index)

# Bike skims
estDF['BikeTime'] = pd.DataFrame(data = bikeSkimFile['Time'][:][origIdxs, destIdxs], index = estDF.index)
estDF['BikeDist'] = pd.DataFrame(data = bikeSkimFile['Distance'][:][origIdxs, destIdxs], index = estDF.index)

# Public transit skims
estDF['PT_TotalTime'] = pd.DataFrame(data = ptwalkSkimFile['Total Time'][:][origIdxs, destIdxs], index = estDF.index)
estDF['Fare'] = pd.DataFrame(data = ptwalkSkimFile['Fare'][:][origIdxs, destIdxs], index = estDF.index)
estDF['AccessWalkTime'] = pd.DataFrame(data = ptwalkSkimFile['Access Walk Time'][:][origIdxs, destIdxs], index = estDF.index)
estDF['EgressWalkTime'] = pd.DataFrame(data = ptwalkSkimFile['Egress Walk Time'][:][origIdxs, destIdxs], index = estDF.index)
estDF['PT_IVTT'] = pd.DataFrame(data = ptwalkSkimFile['In-Vehicle Time'][:][origIdxs, destIdxs], index = estDF.index)
estDF['InitWaitTime'] = pd.DataFrame(data = ptwalkSkimFile['Initial Wait Time'][:][origIdxs, destIdxs], index = estDF.index)
estDF['N_Transfers'] = pd.DataFrame(data = ptwalkSkimFile['Number of Transfers'][:][origIdxs, destIdxs], index = estDF.index)


C:\Users\Kyle\AppData\Local\Temp\ipykernel_48376\1004122564.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  estDF['AutoTime'] = pd.DataFrame(data = autoSkimFile['Time'][:][origIdxs, destIdxs], index = estDF.index)
C:\Users\Kyle\AppData\Local\Temp\ipykernel_48376\1004122564.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  estDF['DriveDist'] = pd.DataFrame(data = autoSkimFile['Distance'][:][origIdxs, destIdxs], index = estDF.index)
C:\Users\Kyle\AppData\Local\Temp\ipykernel_48376\1004122564.py:6: Setti

In [244]:
estDF.head(5)

,SAMPN,weight,visitor_type,party,deptime,arrtime,mode,o_purp,d_purp,o_taz,d_taz,o_ptype,d_ptype,pa_flag,p_taz,a_taz,iz_flag,p_purp,a_purp,trip_purp,auto_time,auto_distance,bike_time,bike_distance,walk_time,walk_distance,bus_gen_cost,bus_fare,bus_ivtt,bus_init_wait,bus_xfer_wait,bus_init_pen_time,bus_xfer_pen_time,bus_xfer_walk_time,bus_access_walk_time,bus_egress_walk_time,bus_dwell_time,bus_total_time,bus_num_xfers,bus_iv_distance,bus_length,bus_walk_time,IZ,business_flag,OTAZIndex,DTAZIndex,AutoTime,DriveDist,WalkTime,WalkDist,BikeTime,BikeDist,PT_TotalTime,Fare,AccessWalkTime,EgressWalkTime,PT_IVTT,InitWaitTime,N_Transfers
0,1000149,34.41,personal,1,9/1/2011 9:00:00,9/1/2011 10:00:00,auto,home,shop,659,400,home,other,1,659,400,0,home,shop,HBShop,22.208900,15.986400,76.92430,15.384900,369.23700,15.384900,38.7878,2.04000,40.70500,2.50000,12.5,0.0,10.0,0.0,5.95639,7.54887,14.5000,93.7102,2.0,16.60890,17.17160,412.1190,0,0,655,396,22.208878,15.986408,369.236603,15.384860,76.924324,15.384860,9.397543e+01,2.040000e+00,5.956390e+00,7.548872e+00,4.097017e+01,2.500000e+00,2.000000e+00
1,1000149,34.41,personal,1,9/1/2011 13:00:00,9/1/2011 14:30:00,auto,shop,recreation,400,33,other,other,1,400,33,0,shop,recreation,NHB,24.430500,18.141800,89.40520,17.881000,429.14500,17.881000,54.7698,1.18164,57.58180,8.11475,15.0,0.0,10.0,0.0,7.54887,13.31840,25.2869,136.8510,2.0,28.71360,29.58310,709.9940,0,0,396,32,24.430458,18.141794,429.144775,17.881035,89.405190,17.881035,1.368728e+02,1.181639e+00,7.548872e+00,1.331844e+01,5.760382e+01,8.114754e+00,2.000000e+00
2,1000149,34.41,personal,1,9/1/2011 16:00:00,9/1/2011 17:00:00,auto,recreation,home,33,659,other,home,0,659,33,0,home,recreation,HBRec,6.684790,2.611840,13.05920,2.611840,62.68420,2.611840,21.6996,0.68000,8.18236,5.00000,0.0,0.0,0.0,0.0,7.41075,14.84370,8.0000,43.4368,0.0,2.34968,3.27695,78.6467,0,0,32,655,7.174065,3.069904,73.677696,3.069904,15.349521,3.069904,4.480818e+01,6.800000e-01,1.297353e+01,7.410750e+00,8.423898e+00,7.500000e+00,0.000000e+00
3,1000150,68.82,personal,1,9/1/2011 7:00:00,9/1/2011 11:00:00,tnc,home,recreation,1106,1106,other,other,1,1106,1106,1,home,recreation,HBRec,0.716316,0.358312,1.79156,0.358312,8.59949,0.358312,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,1102,1102,0.716316,0.358312,8.599492,0.358312,1.791561,0.358312,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38,-3.402823e+38
4,1000150,68.82,personal,1,9/1/2011 15:00:00,9/1/2011 16:00:00,tnc,recreation,home,1106,636,other,other,0,636,1106,0,home,recreation,HBRec,25.785600,14.510700,71.65790,14.331600,343.95800,14.331600,44.2996,1.36000,30.24320,15.00000,10.0,0.0,5.0,0.0,3.13801,15.59080,20.0000,98.9720,1.0,14.52620,15.30650,367.3570,0,0,1102,632,25.694101,14.585185,344.739899,14.364161,71.820839,14.364161,9.360126e+01,6.800000e-01,1.559076e+01,4.588924e+00,2.936497e+01,6.509433e+00,1.000000e+00


In [245]:
estDF['AutoTime'] = estDF['AutoTime'].apply(lambda x: 9999 if np.isclose(x, -3.40285e+38, atol = 100) else x)
estDF['DriveDist'] = estDF['DriveDist'].apply(lambda x: 9999 if np.isclose(x, -3.40285e+38, atol = 100) else x)
estDF['WalkTime'] = estDF['WalkTime'].apply(lambda x: 9999 if np.isclose(x, -3.40285e+38, atol = 100) else x)
estDF['WalkDist'] = estDF['WalkDist'].apply(lambda x: 9999 if np.isclose(x, -3.40285e+38, atol = 100) else x)
estDF['BikeTime'] = estDF['BikeTime'].apply(lambda x: 9999 if np.isclose(x, -3.40285e+38, atol = 100) else x)
estDF['BikeDist'] = estDF['BikeDist'].apply(lambda x: 9999 if np.isclose(x, -3.40285e+38, atol = 100) else x)
estDF['PT_TotalTime'] = estDF['PT_TotalTime'].apply(lambda x: 9999 if np.isclose(x, -3.40285e+38, atol = 100) else x)
estDF['PT_IVTT'] = estDF['PT_IVTT'].apply(lambda x: 9999 if np.isclose(x, -3.40285e+38, atol = 100) else x)
estDF['Fare'] = estDF['Fare'].apply(lambda x: 9999 if np.isclose(x, -3.40285e+38, atol = 100) else x)
estDF['InitWaitTime'] = estDF['InitWaitTime'].apply(lambda x: 9999 if np.isclose(x, -3.40285e+38, atol = 100) else x)
estDF['N_Transfers'] = estDF['N_Transfers'].apply(lambda x: 9999 if np.isclose(x, -3.40285e+38, atol = 100) else x)
estDF['AccessWalkTime'] = estDF['AccessWalkTime'].apply(lambda x: 9999 if np.isclose(x, -3.40285e+38, atol = 100) else x)
estDF['EgressWalkTime'] = estDF['EgressWalkTime'].apply(lambda x: 9999 if np.isclose(x, -3.40285e+38, atol = 100) else x)

C:\Users\Kyle\AppData\Local\Temp\ipykernel_48376\1836316654.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  estDF['AutoTime'] = estDF['AutoTime'].apply(lambda x: 9999 if np.isclose(x, -3.40285e+38, atol = 100) else x)
C:\Users\Kyle\AppData\Local\Temp\ipykernel_48376\1836316654.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  estDF['DriveDist'] = estDF['DriveDist'].apply(lambda x: 9999 if np.isclose(x, -3.40285e+38, atol = 100) else x)
C:\Users\Kyle\AppData\Local\Temp\ipykernel_48376\1836316654.py:3: 

In [246]:
# Add relevant HH fields
# estDF = pd.merge(estDF, hhDF[['ID', 'HHSize', 'Vehicles', 'Income', 'CommercialVehicles', 'WorkersEmployed']], left_on = 'HouseholdID', right_on = 'ID')
# estDF.head()

In [247]:
# Attach relevant TAZ demographics
# First on origins
estDF = pd.merge(estDF, tazDF[['TAZ', 'AreaType']], left_on = 'o_taz', right_on = 'TAZ')
estDF.rename(columns={'AreaType':'O_AreaType'}, inplace=True)
estDF.drop(['TAZ'], axis=1, inplace=True)
estDF.head()

,SAMPN,weight,visitor_type,party,deptime,arrtime,mode,o_purp,d_purp,o_taz,d_taz,o_ptype,d_ptype,pa_flag,p_taz,a_taz,iz_flag,p_purp,a_purp,trip_purp,auto_time,auto_distance,bike_time,bike_distance,walk_time,walk_distance,bus_gen_cost,bus_fare,bus_ivtt,bus_init_wait,bus_xfer_wait,bus_init_pen_time,bus_xfer_pen_time,bus_xfer_walk_time,bus_access_walk_time,bus_egress_walk_time,bus_dwell_time,bus_total_time,bus_num_xfers,bus_iv_distance,bus_length,bus_walk_time,IZ,business_flag,OTAZIndex,DTAZIndex,AutoTime,DriveDist,WalkTime,WalkDist,BikeTime,BikeDist,PT_TotalTime,Fare,AccessWalkTime,EgressWalkTime,PT_IVTT,InitWaitTime,N_Transfers,O_AreaType
0,1000149,34.41,personal,1,9/1/2011 9:00:00,9/1/2011 10:00:00,auto,home,shop,659,400,home,other,1,659,400,0,home,shop,HBShop,22.208900,15.986400,76.924300,15.384900,369.23700,15.384900,38.78780,2.04,40.705000,2.5000,12.5,0.0,10.0,0.0,5.95639,7.54887,14.5,93.7102,2.0,16.608900,17.171600,412.1190,0,0,655,396,22.208878,15.986408,369.236603,15.384860,76.924324,15.384860,93.975433,2.04,5.95639,7.548872,40.970169,2.5,2.0,Downtown
1,1000161,204.39,personal,2,9/1/2011 15:45:00,9/1/2011 16:00:00,walk,eat,home,659,637,other,home,0,637,659,0,home,eat,HBEat,2.557440,0.977263,4.886310,0.977263,23.45430,0.977263,14.19430,0.68,2.007110,2.5000,0.0,0.0,0.0,0.0,11.58000,5.95639,3.0,25.0435,0.0,0.585406,1.316090,31.5861,0,0,655,633,1.701412,0.677653,16.263664,0.677653,3.388264,0.677653,25.543495,0.68,5.95639,11.579997,2.007107,2.5,0.0,Downtown
2,1000167,116.79,personal,2,9/1/2011 11:00:00,9/1/2011 11:10:00,walk,shop,home,659,672,other,home,0,672,659,0,home,shop,HBShop,1.326250,0.537498,2.687490,0.537498,12.90000,0.537498,7.43654,0.68,0.825759,2.2093,0.0,0.0,0.0,0.0,1.57743,5.95639,2.0,12.5689,0.0,0.240846,0.554756,13.3141,0,0,655,668,1.744900,0.715380,17.169113,0.715380,3.576898,0.715380,12.359583,0.68,5.95639,1.577434,0.825759,2.0,0.0,Downtown
3,1000167,116.79,personal,6,9/1/2011 19:30:00,9/1/2011 19:45:00,walk,eat,home,659,672,other,home,0,672,659,0,home,eat,HBEat,1.326250,0.537498,2.687490,0.537498,12.90000,0.537498,7.43654,0.68,0.825759,2.2093,0.0,0.0,0.0,0.0,1.57743,5.95639,2.0,12.5689,0.0,0.240846,0.554756,13.3141,0,0,655,668,1.744900,0.715380,17.169113,0.715380,3.576898,0.715380,12.359583,0.68,5.95639,1.577434,0.825759,2.0,0.0,Downtown
4,1000183,49.15,personal,1,9/1/2011 7:00:00,9/1/2011 10:00:00,walk,home,business,659,659,other,other,1,659,659,1,home,business,HBW,0.391693,0.177754,0.888772,0.177754,4.26611,0.177754,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,655,655,0.391693,0.177754,4.266106,0.177754,0.888772,0.177754,9999.000000,9999.00,9999.00000,9999.000000,9999.000000,9999.0,9999.0,Downtown


In [248]:
# Then on destinations
estDF = pd.merge(estDF, tazDF[['TAZ', 'AreaType']], left_on = 'd_taz', right_on = 'TAZ')
estDF.rename(columns={'AreaType':'D_AreaType'}, inplace=True)
estDF.drop(['TAZ'], axis=1, inplace=True)
estDF.head()

,SAMPN,weight,visitor_type,party,deptime,arrtime,mode,o_purp,d_purp,o_taz,d_taz,o_ptype,d_ptype,pa_flag,p_taz,a_taz,iz_flag,p_purp,a_purp,trip_purp,auto_time,auto_distance,bike_time,bike_distance,walk_time,walk_distance,bus_gen_cost,bus_fare,bus_ivtt,bus_init_wait,bus_xfer_wait,bus_init_pen_time,bus_xfer_pen_time,bus_xfer_walk_time,bus_access_walk_time,bus_egress_walk_time,bus_dwell_time,bus_total_time,bus_num_xfers,bus_iv_distance,bus_length,bus_walk_time,IZ,business_flag,OTAZIndex,DTAZIndex,AutoTime,DriveDist,WalkTime,WalkDist,BikeTime,BikeDist,PT_TotalTime,Fare,AccessWalkTime,EgressWalkTime,PT_IVTT,InitWaitTime,N_Transfers,O_AreaType,D_AreaType
0,1000149,34.41,personal,1,9/1/2011 9:00:00,9/1/2011 10:00:00,auto,home,shop,659,400,home,other,1,659,400,0,home,shop,HBShop,22.20890,15.986400,76.92430,15.384900,369.2370,15.384900,38.7878,2.04,40.70500,2.5,12.5,0.0,10.0,0.0,5.95639,7.54887,14.5,93.7102,2.0,16.608900,17.17160,412.1190,0,0,655,396,22.208878,15.986408,369.236603,15.384860,76.924324,15.384860,93.975433,2.04,5.956390,7.548872,40.970169,2.5,2.0,Downtown,Urban
1,1001269,83.41,personal,4,1/1/2000 18:00:00,1/1/2000 18:01:00,auto,home,shop,636,400,home,other,1,636,400,0,home,shop,HBShop,20.70230,15.462700,74.37040,14.874100,356.9780,14.874100,36.4627,1.36,34.63790,15.0,10.0,0.0,5.0,0.0,3.13801,4.29225,15.0,87.0682,1.0,20.450900,20.76050,498.2520,0,0,632,396,20.702343,15.462737,356.977509,14.874064,74.370354,14.874064,87.094872,1.36,3.138008,4.292253,34.664612,15.0,1.0,Downtown,Urban
2,1000755,34.61,personal,1,1/1/2000 12:30:00,1/1/2000 12:31:00,auto,recreation,eat,1,400,other,other,1,1,400,0,recreation,eat,NHB,28.45300,20.149800,92.39840,18.479700,443.5120,18.479700,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,396,28.710817,20.149845,443.512207,18.479675,92.398376,18.479675,9999.000000,9999.00,9999.000000,9999.000000,9999.000000,9999.0,9999.0,Rural,Urban
3,1000161,204.39,personal,2,9/1/2011 15:45:00,9/1/2011 16:00:00,walk,eat,home,659,637,other,home,0,637,659,0,home,eat,HBEat,2.55744,0.977263,4.88631,0.977263,23.4543,0.977263,14.1943,0.68,2.00711,2.5,0.0,0.0,0.0,0.0,11.58000,5.95639,3.0,25.0435,0.0,0.585406,1.31609,31.5861,0,0,655,633,1.701412,0.677653,16.263664,0.677653,3.388264,0.677653,25.543495,0.68,5.956390,11.579997,2.007107,2.5,0.0,Downtown,Downtown
4,1000190,18.08,business,1,9/1/2011 9:30:00,9/1/2011 9:39:00,walk,recreation,home,659,637,other,home,0,637,659,0,home,recreation,HBRec,2.55744,0.977263,4.88631,0.977263,23.4543,0.977263,14.1943,0.68,2.00711,2.5,0.0,0.0,0.0,0.0,11.58000,5.95639,3.0,25.0435,0.0,0.585406,1.31609,31.5861,0,1,655,633,1.701412,0.677653,16.263664,0.677653,3.388264,0.677653,25.543495,0.68,5.956390,11.579997,2.007107,2.5,0.0,Downtown,Downtown


In [249]:
# Attach the relevant TAZ accessibilities
# First on origins
# estDF = pd.merge(estDF, accDF[['ZONE', 'TransitStopDensity', 'WalkIndex', 'JobsHousingDiversity', 'JobMixDiversity', 'TransitAccessibilityToJobsAM']], left_on = 'HHTAZ', right_on = 'ZONE')
# estDF.rename(columns={'TransitStopDensity':'O_TransitStopDensity', 'WalkIndex':'O_WalkIndex', 'JobsHousingDiversity':'O_JobsHousingDiversity', 'JobMixDiversity':'O_JobMixDiversity', 'TransitAccessibilityToJobsAM': 'O_TransitAccessibilityToJobsAM'}, inplace=True)
# estDF.drop(['ZONE'], axis=1, inplace=True)
# estDF.head()

In [250]:
# Then on destinations
# estDF = pd.merge(estDF, accDF[['ZONE', 'TransitStopDensity', 'WalkIndex', 'JobsHousingDiversity', 'JobMixDiversity', 'TransitAccessibilityToJobsAM']], left_on = 'WorkTAZ', right_on = 'ZONE')
# estDF.rename(columns={'TransitStopDensity':'D_TransitStopDensity', 'WalkIndex':'D_WalkIndex', 'JobsHousingDiversity':'D_JobsHousingDiversity', 'JobMixDiversity':'D_JobMixDiversity', 'TransitAccessibilityToJobsAM': 'D_TransitAccessibilityToJobsAM'}, inplace=True)
# estDF.drop(['ZONE'], axis=1, inplace=True)
# estDF.head()

In [251]:
# Drop records when PT was chosen but not available
estDF = estDF[~((estDF["mode"] == "bus") & (estDF["PT_IVTT"] == 9999))]

In [252]:
estDF.describe()

,SAMPN,weight,party,o_taz,d_taz,pa_flag,p_taz,a_taz,iz_flag,auto_time,auto_distance,bike_time,bike_distance,walk_time,walk_distance,bus_gen_cost,bus_fare,bus_ivtt,bus_init_wait,bus_xfer_wait,bus_init_pen_time,bus_xfer_pen_time,bus_xfer_walk_time,bus_access_walk_time,bus_egress_walk_time,bus_dwell_time,bus_total_time,bus_num_xfers,bus_iv_distance,bus_length,bus_walk_time,IZ,business_flag,OTAZIndex,DTAZIndex,AutoTime,DriveDist,WalkTime,WalkDist,BikeTime,BikeDist,PT_TotalTime,Fare,AccessWalkTime,EgressWalkTime,PT_IVTT,InitWaitTime,N_Transfers
count,4.329000e+03,4329.000000,4329.000000,4329.000000,4329.00000,4329.000000,4329.000000,4329.000000,4329.000000,4291.000000,4291.000000,4291.000000,4291.000000,4291.000000,4291.000000,3418.000000,3418.000000,3418.000000,3418.000000,3418.000000,3418.0,3418.000000,3418.000000,3418.000000,3418.000000,3418.000000,3418.000000,3418.000000,3418.000000,3418.000000,3418.000000,4329.000000,4329.000000,4329.000000,4329.000000,4329.000000,4329.000000,4329.000000,4329.000000,4329.000000,4329.000000,4329.000000,4329.000000,4329.000000,4329.000000,4329.000000,4329.000000,4329.000000
mean,1.001398e+06,102.217253,2.061446,636.268422,636.53569,0.701086,640.410025,632.394086,0.157311,8.316190,5.143310,24.825539,4.965108,119.162585,4.965108,20.367561,0.747790,13.783120,5.760917,2.376468,0.0,1.335638,0.292455,7.037922,7.322137,6.365123,44.273778,0.267128,6.115608,6.726131,161.427035,0.157311,0.143682,632.528529,632.795796,8.474633,5.217364,120.995856,5.041494,25.207470,5.041494,2118.618812,2084.004578,2089.355806,2088.913044,2094.317412,2088.203030,2083.622488
std,5.929055e+02,91.824469,9.718825,222.966743,222.33918,0.457835,205.969124,238.103970,0.364136,12.802450,8.946637,43.074535,8.614907,206.757774,8.614907,18.011601,0.220095,24.656822,4.957087,4.843052,0.0,2.643998,1.129230,8.079029,7.646903,7.875251,46.966959,0.528800,12.110940,12.419877,298.077045,0.364136,0.350808,222.316828,221.687417,13.006159,9.007823,208.333064,8.680544,43.402722,8.680544,4043.587012,4061.131497,4058.392413,4058.618586,4055.898864,4058.979936,4061.327567
min,1.000149e+06,7.900000,1.000000,1.000000,1.00000,0.000000,1.000000,1.000000,0.000000,0.114406,0.066737,0.333685,0.066737,1.601690,0.066737,4.482780,0.680000,0.167034,2.000000,0.000000,0.0,0.000000,0.000000,0.929531,0.675964,0.500000,6.778900,0.000000,0.048718,0.235859,5.660610,0.000000,0.000000,0.000000,0.000000,0.114406,0.066737,1.601686,0.066737,0.333685,0.066737,6.778905,0.680000,0.868469,0.675964,0.167034,2.000000,0.000000
25%,1.000907e+06,49.150000,1.000000,636.000000,636.00000,0.000000,636.000000,633.000000,0.000000,1.079100,0.490982,2.454910,0.490982,11.783600,0.490982,7.846840,0.680000,0.825759,2.244900,0.000000,0.0,0.000000,0.000000,2.138350,2.796000,1.000000,12.611800,0.000000,0.240846,0.595706,14.296900,0.000000,0.000000,632.000000,632.000000,1.079098,0.474427,11.382570,0.474274,2.371369,0.474274,14.458420,0.680000,3.088051,2.900010,1.068328,2.244898,0.000000
50%,1.001463e+06,79.360000,1.000000,664.000000,664.00000,1.000000,667.000000,659.000000,0.000000,2.604710,1.114420,5.572090,1.114420,26.746000,1.114420,12.560200,0.680000,2.466160,2.500000,0.000000,0.0,0.000000,0.000000,3.838430,5.592800,3.000000,22.627400,0.000000,0.774643,1.316090,31.586100,0.000000,0.000000,660.000000,660.000000,2.557441,1.075626,25.720434,1.071685,5.358424,1.071685,34.155407,0.680000,5.956390,5.956390,5.933113,6.500000,0.000000
75%,1.001908e+06,117.670000,2.000000,680.000000,680.00000,1.000000,676.000000,680.000000,0.000000,8.793510,4.333170,20.187000,4.037390,96.897400,4.037390,26.602475,0.680000,14.192125,7.500000,0.000000,0.0,0.000000,0.000000,7.410750,7.410750,8.500000,57.989925,0.000000,5.154005,5.921473,142.115250,0.000000,0.000000,676.000000,676.000000,9.035065,4.515799,100.607285,4.191970,20.959854,4.191970,146.873825,1.360000,25.354368,25.354368,59.872807,15.000000,1.000000
max,1.002349e+06,642.340000,360.000000,1116.000000,1116.00000,1.000000,1116.000000,1116.000000,1.000000,86.170

In [253]:
# Larch requires the chosen options be integers
conditions = [
    estDF['mode'] == 'auto',
    estDF['mode'] == 'bus',
    estDF['mode'] == 'tnc',
    estDF['mode'] == 'walk',
    estDF['mode'] == 'bike'
]

values = [1, 2, 3, 4, 5]

estDF['mode'] = np.select(conditions, values, default = -1)

# Finally, filter the estimation table to just the current trip purpose
# 'HBShop', 'NHB', 'HBEat', 'HBRec', 'HBO', 'HBW'
estDF = estDF[(estDF['trip_purp'] == 'HBW')]

In [254]:
# Create Larch Dataframe from selected data and instantiate larch model using data
auto = 1
bus = 2
tnc = 3
walk = 4
bike = 5

dfs = larch.DataFrames(co = estDF, 
                       alt_codes = [auto, bus, tnc, walk, bike], 
                       alt_names = ['auto', 'bus', 'tnc', 'walk', 'bike'], 
                       ch_name = 'mode',
                       av = 1)

# Create Larch Model
m = larch.Model(dataservice=dfs)

# Define Title
m.title = "Oahu Visitor Mode Choice - All Trips"

# Choice
m.choice_co_code = 'mode'

# Availability

In [255]:
m.utility_co[auto] = (
    + P.ASC_auto
    + P.Time * X('AutoTime')
    # + P.BusiFlag * X('business_flag')
)

# m.utility_co[bus] = (
#     + P.ASC_bus
#     + P.Time * X('PT_IVTT')
#     + P.Fare * X('Fare')
#     # + P.Transfers * X('N_Transfers')
#     # + P.InitialWaitTime * X('InitWaitTime')
#     # + P.AccessEgressTimes * X('AccessWalkTime + EgressWalkTime')
#     + P.AccessEgressInitWaitTimes * X('AccessWalkTime + InitWaitTime + EgressWalkTime')
# )

m.utility_co[tnc] = (
    + P.Time * X('AutoTime')
    # + P.BusiFlag * X('business_flag')
)

m.utility_co[walk] = (
    + P.ASC_walk
    + P.Time * X('WalkTime')
    # + P.Time_NM * X('WalkTime')
    # + P.IZ * X('IZ')
    # + P.CBD * X('O_AreaType == "Downtown"') * X('D_AreaType == "Downtown"')
)

# m.utility_co[bike] = (
#     + P.ASC_bike
#     + P.Time_NM * X('BikeTime')
#     + P.CBD * X('O_AreaType == "Downtown"') * X('D_AreaType == "Downtown"')
#     # + P.IZ * X('IZ')
#     # + P.JobsHousingDiversity * X('O_JobsHousingDiversity')
#     # + P.JobMixDiversity * X('O_JobMixDiversity')
# )

# m.lock_values(Time = -0.025)

In [256]:
m.load_data()

req_data does not request avail_ca or avail_co but it is set and being provided


In [257]:
m.dataframes.choice_avail_summary()

,name,chosen,available
1,auto,53.0,117.0
2,bus,0.0,117.0
3,tnc,20.0,117.0
4,walk,44.0,117.0
5,bike,0.0,117.0
< Total All Alternatives >,,117.0,


In [258]:
m.dataframes.data_co.statistics()

AutoTime,117,0.21319,64.2049,3.23534,"<div class=""figure"" metadata=""gAWVkAEAAAAAAAB9lCiMC2Jpbl9oZWlnaHRzlIwSbnVtcHkuY29yZS5udW1lcmljlIwLX2Zyb21idWZmZXKUk5QolkgAAAAAAAAAQgAAAAAAAAAPAAAAAAAAABAAAAAAAAAABgAAAAAAAAAIAAAAAAAAAAQAAAAAAAAAAAAAAAAAAAABAAAAAAAAAAEAAAAAAAAAlIwFbnVtcHmUjAVkdHlwZZSTlIwCaTiUiYiHlFKUKEsDjAE8lE5OTkr/////Sv////9LAHSUYksJhZSMAUOUdJRSlIwJYmluX2VkZ2VzlGgEKJZQAAAAAAAAAAAAAIDNScs/5DiOQyNLHUDkOI4N/N0sQKuqqjwzizVA5DiOcminPECO4zjUzuFBQKuqKm/pb0VAyHEcCgT+SEDkOA6lHoxMQAAAAKAcDVBAlGgIjAJmOJSJiIeUUpQoSwNoDE5OTkr/////Sv////9LAHSUYksKhZRoD3SUUpSMEXplcm9zX2FyZV9kcm9wcGVklImMEnJhbmdlX2lzX3RydW5jYXRlZJRLAHUu"" zeros_are_dropped=""False"" range_is_truncated=""0"" style=""min-width:Truein""> 2023-08-23T17:10:30.863205 image/svg+xml Matplotlib v3.7.2, https://matplotlib.org/",10.9625,13.0222,0,117,0,0.21319,64.2049,10.9625,13.0222
WalkTime,117,2.78497,944.996,37.4695,"<div class=""figure"" metadata=""gAWVkAEAAAAAAAB9lCiMC2Jpbl9oZWlnaHRzlIwSbnVtcHkuY29yZS5udW1lcmljlIwLX2Zyb21idWZmZXKUk5QolkgAAAAAAAAARgAAAAAAAAALAAAAAAAAABAAAAAAAAAABAAAAAAAAAAEAAAAAAAAAAYAAAAAAAAABAAAAAAAAAAAAAAAAAAAAAIAAAAAAAAAlIwFbnVtcHmUjAVkdHlwZZSTlIwCaTiUiYiHlFKUKEsDjAE8lE5OTkr/////Sv////9LAHSUYksJhZSMAUOUdJRSlIwJYmluX2VkZ2VzlGgEKJZQAAAAAAAAAAAAAMCgRwZAVVVVBWfeWkBVVVWCSIVqQAAAAMGuzXNAVVXVQLlYekBVVVXg4XGAQAAAQCBnt4NAqqoqYOz8hkBVVRWgcUKKQAAAAOD2h41AlGgIjAJmOJSJiIeUUpQoSwNoDE5OTkr/////Sv////9LAHSUYksKhZRoD3SUUpSMEXplcm9zX2FyZV9kcm9wcGVklImMEnJhbmdlX2lzX3RydW5jYXRlZJRLAHUu"" zeros_are_dropped=""False"" range_is_truncated=""0"" style=""min-width:Truein""> 2023-08-23T17:10:30.893857 image/svg+xml Matplotlib v3.7.2, https://matplotlib.org/",160.351,212.683,0,117,0,2.78497,944.996,160.351,212.683


In [259]:
# Run MLE
m.set_cap(cap=100.0)
m.maximize_loglike(method='SLSQP', maxiter=1000)

# Order output parameters
m.ordering = [
        ("ASCs", 'ASC.*',),
        ("LOS", 'B_.*',),
]

# Calculate Stats
m.calculate_parameter_covariance()

,value,initvalue,nullvalue,minimum,maximum,holdfast,note,best
ASC_auto,2.391825,0.0,0.0,-100.0,100.0,0,,2.391825
ASC_walk,3.683117,0.0,0.0,-100.0,100.0,0,,3.683117
Time,-0.025300,0.0,0.0,-100.0,100.0,0,,-0.025300


In [260]:
m.parameter_summary()

In [261]:
m.estimation_statistics()

<xmle.Elem 'div' with 1 children>

In [209]:
# Report
report = larch.Reporter(title=m.title)
report << '## Data Statistics' << m.dataframes.data_co.statistics()
report << '## Choice Table Summary' << m.dataframes.choice_avail_summary()
report << "## Utility Functions" << m.utility_functions()
report << '## Parameter Summary' << m.parameter_summary()
report << "## Estimation Statistics" << m.estimation_statistics()
report.save(
    '.\Oahu_VisMC_HBW.html',
    overwrite=True,
    metadata=m,
)

'.\\Oahu_VisMC_HBW.html'